In [1]:
!pip install transformers evaluate datasets

In [2]:
!pip install accelerate -U

In [3]:
from google.colab import drive

drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
import random
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import TfidfVectorizer # baseline code용 tfidef vectorirzer(또는 tokenizer)
from sklearn import preprocessing
from sklearn.metrics import f1_score
from IPython.display import display, HTML

from transformers import AutoModel, AutoTokenizer,AutoModelForSequenceClassification, TrainingArguments, Trainer # 사용하고자 하는 모델, 토크나이저 적용시 필요
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset, ClassLabel
from datasets import load_dataset, ClassLabel, load_metric

from tqdm.auto import tqdm # process bar 표시용

import warnings
warnings.filterwarnings(action='ignore')

import urllib.request

In [35]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

device

device(type='cpu')

In [36]:
pretrained_model = 'klue/roberta-small'
tokenizer = AutoTokenizer.from_pretrained(pretrained_model)

In [37]:
accuracy = load_metric("accuracy")
def compute_metrics(eval_pred) :
  predictions, labels = eval_pred
  predictions = np.argmax(predictions, axis = 1)

  return accuracy.compute(predictions = predictions, references = labels)

id2label = {0:'부정', 1:'긍정'}
label2id = {'부정':0, '긍정':1}

In [38]:
model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/nlpbook/best_model/my_model')

model.eval()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (

In [39]:
def inference_fn(sentence) :
  inputs = tokenizer(
      [sentence],
      max_length = 20,
      padding = 'max_length',
      truncation = True,
  )
  with torch.no_grad() :
    outputs = model(**{k: torch.tensor(v) for k, v in inputs.items()})
    prob = outputs.logits.softmax(dim = 1)
    positive_prob = round(prob[0][1].item(), 4)
    negative_prob = round(prob[0][0].item(), 4)
    pred = '긍정 (positive)' if torch.argmax(prob) == 1 else '부정 (negative)'
  return {
      'sentence' : sentence,
      'prediction' : pred,
      'positive_data' : f'긍정 {positive_prob}',
      'negative_data' : f'부정 {negative_prob}',
      'positive_width' : f'{positive_prob * 100}%',
      'negative_width' : f'{round(negative_prob * 100, 2)}%',
  }

In [40]:
sentence = '기린 그린 기린 그림이 진짜 기린 그린 기린 그림인지 아닌 기린 그린 그림인지 헷갈린다!'
inference_fn(sentence)

{'sentence': '기린 그린 기린 그림이 진짜 기린 그린 기린 그림인지 아닌 기린 그린 그림인지 헷갈린다!',
 'prediction': '부정 (negative)',
 'positive_data': '긍정 0.0305',
 'negative_data': '부정 0.9695',
 'positive_width': '3.05%',
 'negative_width': '96.95%'}

In [41]:
sentence = '맞다!'
inference_fn(sentence)

{'sentence': '맞다!',
 'prediction': '긍정 (positive)',
 'positive_data': '긍정 0.6249',
 'negative_data': '부정 0.3751',
 'positive_width': '62.49%',
 'negative_width': '37.51%'}

In [42]:
# 로컬 폴더를 원격에서 접근 가능한 터널로 만들어주는 작업을 수행
# ngrok의 구성 파일과 관련된 파일들을 저장 authtoken 이라는 설정값을 가진 ngrok.yml 파일을 생성하고 인증 토큰 값을 설정
!mkdir /root/.ngrok2 && echo 'authtoken: 2T5dj5MGKLVe9gdeTmRn96rqUGK_6bLTw98KhQnQENd1C9FCt' > /root/.ngrok2/ngrok.yml

mkdir: cannot create directory ‘/root/.ngrok2’: File exists


In [43]:
!ls /root -al

total 72
drwx------ 1 root root 4096 Aug  2 02:51 .
drwxr-xr-x 1 root root 4096 Aug  2 02:22 ..
-r-xr-xr-x 1 root root 1169 Jan  1  2000 .bashrc
drwxr-xr-x 1 root root 4096 Aug  2 02:24 .cache
drwx------ 1 root root 4096 Aug  2 02:24 .config
drwxr-xr-x 5 root root 4096 Jul 31 20:18 .ipython
drwx------ 1 root root 4096 Jul 31 20:18 .jupyter
drwxr-xr-x 2 root root 4096 Jul 31 20:16 .keras
drwx------ 3 root root 4096 Jul 31 19:54 .launchpadlib
drwxr-xr-x 1 root root 4096 Jul 31 20:18 .local
drwxr-xr-x 2 root root 4096 Aug  2 02:51 .ngrok2
drwxr-xr-x 4 root root 4096 Jul 31 20:18 .npm
-rw-r--r-- 1 root root  161 Jul  9  2019 .profile
-r-xr-xr-x 1 root root  254 Jan  1  2000 .tmux.conf
-rw-r--r-- 1 root root  165 Jul 31 20:18 .wget-hsts


In [44]:
from flask import Flask, request, jsonify, render_template

def get_web_service_app(inference_fn, is_colab = True) : # colab 환경에서 실행 시 외부에서 접근 가능한 url 생성
  app = Flask(__name__, template_folder = '') # flask 애플리케이션 생성 __name__ 은 현재 모듈의 이름
  if is_colab :
    from flask_ngrok import run_with_ngrok
    run_with_ngrok(app)
  else :
    from flask_cors import CORS
    CORS(app)

  @app.route('/') # 경로에 대한 라우트 정의
  def index() :
    return render_template('index.html') # index.html 파일을 랜더링

  @app.route('/api', methods = ['POST']) # 이 라우트는 post 메서드를 사용
  def api() :
    query_sentence = request.json # api 함수는 클라이언트로부터 json 형식의 데이터를 받아옴
    output_data = inference_fn(query_sentence)
    response = jsonify(output_data) # output 데이터를 json 형식으로 변환하여 클라이언트에게 반환

    return response

  return app # flask 객체를 반환


In [48]:
from google.colab import files

uploaded = files.upload()

Saving index.html to index.html


In [46]:
!ls

drive  index.html  sample_data


In [47]:
# !rm *.html

In [18]:
!pip install flask-ngrok

In [49]:
app = get_web_service_app(inference_fn)

app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://54e1-35-229-149-123.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [02/Aug/2023 03:05:32] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [02/Aug/2023 03:05:32] "GET /favicon.ico HTTP/1.1" 404 -


In [20]:
import shutil

folder_path = '/content/drive/MyDrive/nlpbook/best_model/my_model'
archive_path = '/content/drive/MyDrive/nlpbook/my_archive'
shutil.make_arhive(archive_path, 'zip', folder_path)

AttributeError: ignored